In [3]:
# 划分基因贫瘠区域和基因富集区
# 用着丝粒划分染色体两端，再将染色体端粒区域的30%作为基因富集区，剩下作为基因贫瘠区
import pandas as pd
df_abs = pd.read_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/0DPA_abs.bed",
                     sep='\t', names=['c', 's', 'e', 'binid'])
df_center = pd.read_csv(
    'F:/fiber/airticle/GB_modification/Gene_poor_rich/chrom_Gb.txt', sep='\t')
chrs = ['Gbar_A01', 'Gbar_A02', 'Gbar_A03', 'Gbar_A04', 'Gbar_A05', 'Gbar_A06', 'Gbar_A07', 'Gbar_A08', 'Gbar_A09', 'Gbar_A10', 'Gbar_A11', 'Gbar_A12', 'Gbar_A13',
        'Gbar_D01', 'Gbar_D02', 'Gbar_D03', 'Gbar_D04', 'Gbar_D05', 'Gbar_D06', 'Gbar_D07', 'Gbar_D08', 'Gbar_D09', 'Gbar_D10', 'Gbar_D11', 'Gbar_D12', 'Gbar_D13']


def chr_poor_rich_regions(abs_df,  chr_df):
    # print(11)
    df_abs = abs_df.copy()
    df_chr = chr_df.copy()
    df_abs.reset_index(drop=True, inplace=True)
    df_chr.reset_index(drop=True, inplace=True)
    # print(df_chr)
    center_position = df_chr.loc[0, 'center']
    # print(center_position)
    center_bin = df_abs.query(
        "s<= @center_position & e>=@center_position").index[0]
    df_header = df_abs.loc[0:center_bin, :]
    df_tail = df_abs.loc[center_bin+1:df_abs.shape[0]-1, :]
    df_header.reset_index(drop=True, inplace=True)
    df_tail.reset_index(drop=True, inplace=True)
    header_mid = int(df_header.shape[0]*0.3)
    tail_mid = int(df_tail.shape[0]*0.7)
    df_rich_one = df_header.loc[0:header_mid, :]
    df_poor_one = df_header.loc[header_mid+1:df_header.shape[0]-1, :]
    df_poor_two = df_tail.loc[0:tail_mid, :]
    df_rich_two = df_tail.loc[tail_mid+1: df_tail.shape[0]-1, :]
    return(df_rich_one, df_poor_one, df_poor_two, df_rich_two)


for c in chrs:
    df_abs_c = df_abs.query("c==@c").copy()
    df_chr_c = df_center.query("c==@c").copy()
    df_a, df_b, df_c, df_d = chr_poor_rich_regions(df_abs_c, df_chr_c)
    df_a.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/{0}_rich_one.bed".format(
        c), sep='\t', index=False, header=False)
    df_b.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/{0}_poor_one.bed".format(
        c), sep='\t', index=False, header=False)
    df_c.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/{0}_poor_two.bed".format(
        c), sep='\t', index=False, header=False)
    df_d.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/{0}_rich_two.bed".format(
        c), sep='\t', index=False, header=False)


In [23]:
### 划分基因贫瘠和基因富集区
### 将染色体着丝粒位置做为中心点，切分染色体，再从端粒开始数，占染色体臂长基因总数70%的为基因富集区，剩下的为基因贫瘠区
import pandas as pd
chrs =  ['Gbar_A01', 'Gbar_A02', 'Gbar_A03', 'Gbar_A04', 'Gbar_A05', 'Gbar_A06', 'Gbar_A07', 'Gbar_A08', 'Gbar_A09', 'Gbar_A10', 'Gbar_A11', 'Gbar_A12', 'Gbar_A13','Gbar_D01', 'Gbar_D02', 'Gbar_D03', 'Gbar_D04', 'Gbar_D05', 'Gbar_D06', 'Gbar_D07', 'Gbar_D08', 'Gbar_D09', 'Gbar_D10', 'Gbar_D11', 'Gbar_D12', 'Gbar_D13']
#df_gene = pd.read_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/Gbar_5k_windows_gene.bed", sep='\t', names=['c', 's', 'e', 'genenumber'])
df_gene = pd.read_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/Gbar_10k_windows_gene.bed", sep='\t', names=['c', 's', 'e', 'genenumber'])
df_center = pd.read_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/chrom_Gb.txt", sep='\t')
#df_abs = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/rawdata_5000_abs.bed', sep='\t', names=['c', 's', 'e', 'binid'])
df_abs = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/10k/rawdata_10000_abs.bed', sep='\t', names=['c', 's', 'e', 'binid'])
def poor_rich_region(gene, number, abs):
    df_gene = gene.copy()
    df_abs = abs.copy()
    df_gene.reset_index(drop=True, inplace=True)
    df_abs.reset_index(drop=True, inplace=True)
    gene_sum = 0
    for index, row in df_gene.iterrows():
        gene_sum += row['genenumber']
        if (gene_sum >= number):
            df_tmp = df_gene.loc[0:index, :][['c', 's', 'e']]
            break
    #print(df_tmp)
    #print(df_abs)
    df_one = pd.merge(df_abs, df_tmp, how='inner', on=['c', 's', 'e'])
    df_two = df_abs.append(df_tmp, ignore_index=True)
    df_two.drop_duplicates(keep=False, inplace=True, subset=['c', 's', 'e'])
    df_two['binid'] = df_two['binid'].map(int)
    return df_one, df_two
for c in chrs:
    df_gene_chr = df_gene.query("c==@c")
    df_center_chr = df_center.query("c==@c")
    df_abs_chr = df_abs.query("c==@c")
    df_gene_chr.reset_index(drop=True, inplace=True)
    df_center_chr.reset_index(drop=True, inplace=True)
    df_abs_chr.reset_index(drop=True, inplace=True)
    center = df_center_chr.loc[0, 'center']
    center_bin_index = df_abs_chr.query("s<=@center & e>=@center").index[0]
    one_regions = df_gene_chr.loc[0:center_bin_index, :].copy()
    two_regions = df_gene_chr.loc[center_bin_index+1:df_gene_chr.shape[0]-1, :].copy()
    abs_one_regions = df_abs_chr.loc[0:center_bin_index, :].copy()
    abs_two_regions =  df_abs_chr.loc[center_bin_index+1:df_gene_chr.shape[0]-1, :].copy()
    one_regions_genes = one_regions['genenumber'].sum()
    two_regions_genes = two_regions['genenumber'].sum()
    one_rich_gene = int(one_regions_genes*0.7)
    two_poor_gene = int(two_regions_genes*0.3)
    df_rich_one, df_poor_one = poor_rich_region(one_regions, one_rich_gene, abs_one_regions)
    df_poor_two, df_rich_two = poor_rich_region(two_regions, two_poor_gene, abs_two_regions)
    df_rich_one.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/10k/{0}_rich_one.bed".format(c), sep='\t', index=False, header=False)
    df_poor_one.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/10k/{0}_poor_one.bed".format(c), sep='\t', index=False, header=False)
    df_poor_two.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/10k/{0}_poor_two.bed".format(c), sep='\t', index=False, header=False)
    df_rich_two.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/10k/{0}_rich_two.bed".format(c), sep='\t', index=False, header=False)

In [ ]:
## 划分基因贫瘠和基因富集区
## 将两端异染色质两端做为染色体臂长两端的中点位置
import pandas as pd
df_heter = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/heterochromatin.bed', sep='\t', names=['c', 's', 'e', 'number'])
df_abs = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/rawdata_5000_abs.bed', sep='\t', names=['c', 's', 'e', 'binid'])
df_center = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/chrom_Gb.txt', sep='\t')
#chrs =  ['Gbar_A01', 'Gbar_A02', 'Gbar_A03', 'Gbar_A04', 'Gbar_A05', 'Gbar_A06', 'Gbar_A07', 'Gbar_A08', 'Gbar_A09', 'Gbar_A10', 'Gbar_A11', 'Gbar_A12', 'Gbar_A13','Gbar_D01', 'Gbar_D02', 'Gbar_D03', 'Gbar_D04', 'Gbar_D05', 'Gbar_D06', 'Gbar_D07', 'Gbar_D08', 'Gbar_D09', 'Gbar_D10', 'Gbar_D11', 'Gbar_D12', 'Gbar_D13']
chrs = ['Gbar_A01']
for c in chrs:
    df_heter_chr = df_heter.query('c == @c')
    df_abs_chr = df_heter.query("c == @c")
    df_center_chr = df_center.query("c == @c")
    df_heter_chr.reset_index(drop=True, inplace=True)
    df_abs_chr.reset_index(drop=True, inplace=True)
    df_center_chr.reset_index(drop=True, inplace=True)
    center_position = df_center_chr.loc[0, 'center']
    center_bin = df_abs_chr.query("s<= @center_position & e>=@center_position").index[0]
    ## find mid one
    mid1_position = df_heter_chr.loc[0, 's']
    mid2_position = df_heter_chr.loc[df_center_chr.shape[0]-1, 'e']
    print(df_heter_chr)
    #print(df_abs_chr)
    print(mid1_position , mid2_position)
    mid1 = df_abs_chr.query("s <= @mid1_position & e>= @mid1_position").index[0]
    mid2 = df_abs_chr.query("s <= @mid2_position & e>= @mid2_position").index[0]
    df_rich_one = df_abs_chr.loc[0:mid1, :]
    df_poor_one = df_abs_chr.loc[mid1+1:center_bin, :]
    df_poor_two = df_abs_chr.loc[center_bin+1:mid2, :]
    df_rich_two = df_abs_chr.loc[mid2+1:df_abs_chr.shape[0]-1, :]
    df_rich_one.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/{0}_rich_one.bed".format(c), sep='\t', index=False)
    df_poor_one.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/{0}_poor_one.bed".format(c), sep='\t', index=False)
    df_poor_two.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/{0}_poor_two.bed".format(c), sep='\t', index=False)
    df_rich_two.to_csv("F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/{0}_rich_two.bed".format(c), sep='\t', index=False)

In [ ]:
## choose matrix region
import pandas as pd
import sys
mat_f = sys.argv[1]
abs_f = sys.argv[2]
out_f1 = sys.argv[3]
out_f2 = sys.argv[4]
df_mat = pd.read_csv(mat_f, sep='\t', names=['bin1', 'bin2', 'freq'])
df_abs = pd.read_csv(abs_f, sep='\t', names=['c', 's', 'e', 'binid'])
start_bin = df_abs.loc[0, 'binid']
end_bin = df_abs.loc[df_abs.shape[0]-1, 'binid']
min_bin = start_bin - 1
min_region = df_abs.loc[0, 's']
df_mat_out = df_mat.query("(bin1>=@start_bin & bin1<=@end_bin) & (bin2>=@start_bin & bin1<=@end_bin)").copy()
df_mat_out['bin1'] = df_mat_out['bin1'] - min_bin
df_mat_out['bin2'] = df_mat_out['bin2'] - min_bin
df_abs['binid'] = df_abs['binid'] - min_bin
df_abs['s'] = df_abs['s'] - min_region
df_abs['e'] = df_abs['e'] - min_region
df_mat_out.to_csv(out_f1, sep='\t', index=False, header=False)
df_abs.to_csv(out_f2, sep='\t', index=False, header=False)


In [ ]:
## 批量提交任务
import sys
days = ['0DPA', '5DPA', '10DPA', '20DPA']
chrs = ['Gbar_A01', 'Gbar_A02', 'Gbar_A03', 'Gbar_A04', 'Gbar_A05', 'Gbar_A06', 'Gbar_A07', 'Gbar_A08', 'Gbar_A09', 'Gbar_A10', 'Gbar_A11', 'Gbar_A12', 'Gbar_A13',
        'Gbar_D01', 'Gbar_D02', 'Gbar_D03', 'Gbar_D04', 'Gbar_D05', 'Gbar_D06', 'Gbar_D07', 'Gbar_D08', 'Gbar_D09', 'Gbar_D10', 'Gbar_D11', 'Gbar_D12', 'Gbar_D13']
out_f = sys.argv[1]
fp = open(out_f, 'w')
for d in days:
    for c in chrs:
        for i in ['poor', 'rich']:
            for j in ['one', 'two']:
                inf1 = "./{0}/{0}_5k_iced.matrix".format(d)
                inf2 = "./poor_rich_regions/{0}_{1}_{2}.txt".format(c, i, j)
                out1 = "./{0}/{1}_{2}_{3}.matrix".format(d, c ,i, j)
                out2 = "./{0}/{1}_{2}_{3}_bed.abs".format(d, c, i, j)
                out3 = "./{0}/{1}_{2}_{3}.cool".format(d, c, i, j)
                out4 = "./{0}/{1}_{2}_{3}_pvalues.txt".format(d, c, i, j)
                out5 = "./{0}/{1}_{2}_{3}_SVL.txt".format(d, c, i, j)
                ## split regions
                out_line1 = "bsub -q normal -e %J.err -o %J.out -J choose_region -R span[hosts=1] 'python ~/programs/choose_matrix_regions.py {0} {1} {2} {3}'".format(inf1, inf2, out1, out2)
                ## transformat 
                out_line2 = "bsub -q normal -e %J.err -o %J.out -J transF -R span[hosts=1] 'hicConverFormat -m {0} --bedFileHicpro {1} --inputFormat hicpro --outputFormat cool -o {2}'".format(out1, out2, out3)
                ## SVL
                out_line3 = "bsub -q normal -e %J.err -o %J.out -J SVL -R span[hosts=1] 'hicPlotSVL -m {0} --distance 2000000 --threads 1 --outFilename {1} --outFileNameData {2}' ".format(out3, out4, out5)
                print(out_line1, file=fp)
fp.close()

## 划分基因贫瘠和基因富集区
## 将两端异染色质两端做为染色体臂长两端的中点位置
import pandas as pd
df_heter = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/heterochromatin.bed', sep='\t', names=['c', 's', 'e', 'number'])
df_abs = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/5k/rawdata_5000_abs.bed', sep='\t', names=['c', 's', 'e', 'binid'])
df_center = pd.read_csv('F:/fiber/airticle/GB_modification/Gene_poor_rich/chrom_Gb.txt', sep='\t')
chrs =  ['Gbar_A01', 'Gbar_A02', 'Gbar_A03', 'Gbar_A04', 'Gbar_A05', 'Gbar_A06', 'Gbar_A07', 'Gbar_A08', 'Gbar_A09', 'Gbar_A10', 'Gbar_A11', 'Gbar_A12', 'Gbar_A13','Gbar_D01', 'Gbar_D02', 'Gbar_D03', 'Gbar_D04', 'Gbar_D05', 'Gbar_D06', 'Gbar_D07', 'Gbar_D08', 'Gbar_D09', 'Gbar_D10', 'Gbar_D11', 'Gbar_D12', 'Gbar_D13']

for c in chrs:
    df_heter_chr = df_heter.query('c == @c')
    df_abs_chr = df_heter.query("c == @c")
    df_center_chr = df_center.query("c == @c")
    df_heter_chr.reset_index(drop=True, inplace=True)
    df_abs_chr.reset_index(drop=True